# Evaluating an Azure AI Agent with Azure AI Evaluation SDK

GBB learning sessione example to demonstrate how to evaluate an **Azure AI Agent** using three quality metrics provided by the Azure AI Evaluation SDK (preview):
1. **Intent Resolution** – Did the agent understand and address the user’s request?
2. **Tool Call Accuracy** – Did the agent choose and invoke the correct tool(s) with the right parameters?
3. **Task Adherence** – Did the agent follow its instructions and complete the assigned task?

Created a mock `fetch_weather` tool, simulate an agent response in various scenarios (correct, incorrect, unspecified / right tool chosen, wrong tool chosen) and evaluate with the SDK.

## Setup Azure credentials and project

In [1]:
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv(dotenv_path=".env", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


## Create a sample agent and `fetch_weather` tool

In [3]:
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    FunctionTool,
    ToolSet
)
import json
# from typing import Callable, Any, Set

def fetch_weather(location: str) -> str:
    """Mock weather service"""
    mock_weather_data = {
        'Seattle': 'Sunny, 25°C',
        'London': 'Cloudy, 18°C',
        'Tokyo': 'Rainy, 22°C'
    }
    return json.dumps({'weather': mock_weather_data.get(location, 'N/A')})

def fetch_funfact(location: str) -> str:
    """Fun fact about a location"""
    mock_weather_data = {
        'Seattle': 'There are whales',
        'London': 'Is the capital of England',
        'Tokyo': 'Has the highest population density in the world'
    }
    return json.dumps({'weather': mock_weather_data.get(location, 'N/A')})

# Initialize project client with proper authentication
project_client = AIProjectClient(
    credential=credential,  # Use the credential from earlier setup
    endpoint=settings.project_endpoint
)
print(f"project client api version: {project_client._config.api_version}")
    
# Register functions as tools
# custom_fns: Set[Callable[..., Any]] = {fetch_weather, fetch_funfact}
functions = FunctionTool({fetch_weather, fetch_funfact})
toolset = ToolSet()
toolset.add(functions)
    
# Create agent with proper error handling
AGENT_NAME = settings.agent_name
AGENT_INSTRUCTIONS = """You are a helpful weather assistant. When asked about the weather in a location:
            1. Use fetch_weather to get current conditions
            2. Provide clear, concise responses
            3. Stay focused on weather information
            Always use tools when available and verify data before responding."""

found_agent = None
all_agents_list = project_client.agents.list_agents()
for a in all_agents_list:
    if a.name == AGENT_NAME:
        found_agent = a
        break

project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4)
if found_agent:
    agent = project_client.agents.update_agent(
        agent_id=found_agent.id,
        model=settings.model_deployment_name,
        instructions=AGENT_INSTRUCTIONS,
        toolset=toolset,
    )
    project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4) 
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project_client.agents.create_agent(
        model=settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
        toolset=toolset,
    )
    print(f"Created agent '{AGENT_NAME}' with {len(functions._functions)} tools")

project client api version: 2025-05-15-preview
Created agent 'Weather Assistant' with 2 tools


## Simulate a user query and agent responses

In [4]:
user_question = "What's the weather in Seattle?"
user_question_unspecific = "How much does it rain in Spain?"

# Correct tool usage
import json
weather_seattle = json.loads(fetch_weather('Seattle'))['weather']
weather_london = json.loads(fetch_weather('London'))['weather']
agent_response_correct = (
    f'The current weather in Seattle is {weather_seattle}.'
)

# Incorrect tool usage (wrong location)
agent_response_incorrect = (
    'The weather in Seattle is Rainy, 15°C. In London, it\'s Sunny, 28°C.'
)

print('User:', user_question)
print('Agent (correct):', agent_response_correct)
print('Agent (incorrect):', agent_response_incorrect)

print('\nUser:', user_question_unspecific)
print('Agent without information, providing irrelevant information')

User: What's the weather in Seattle?
Agent (correct): The current weather in Seattle is Sunny, 25°C.
Agent (incorrect): The weather in Seattle is Rainy, 15°C. In London, it's Sunny, 28°C.

User: How much does it rain in Spain?
Agent without information, providing irrelevant information


## Initialize evaluation metrics

In [5]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.ai.evaluation import (
    IntentResolutionEvaluator,
    ToolCallAccuracyEvaluator,
    TaskAdherenceEvaluator,
)

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=settings.azure_openai_endpoint,
    api_key=settings.azure_openai_api_key,
    api_version=settings.azure_openai_api_version,
    azure_deployment=settings.model_deployment_name,
)

intent_eval = IntentResolutionEvaluator(model_config=model_config)
tool_eval = ToolCallAccuracyEvaluator(model_config=model_config)
task_eval = TaskAdherenceEvaluator(model_config=model_config)
print('Evaluators setup')

Class IntentResolutionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ToolCallAccuracyEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class TaskAdherenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluators setup


### Intent Resolution

Measures how well the agent identifies the user’s request, including how well it scopes the user’s intent, asks clarifying questions, and reminds end users of its scope of capabilities.


In [6]:
from pprint import pprint
res_intent_correct = intent_eval(query=user_question, response=agent_response_correct)
res_intent_incorrect = intent_eval(query=user_question, response=agent_response_incorrect)
res_intent_unspecific = intent_eval(query=user_question_unspecific, response=agent_response_incorrect)
print('Correct:')
pprint(res_intent_correct, width=120, compact=True)
print('\nIncorrect:')
pprint(res_intent_incorrect, width=120, compact=True)
print('\nUnspecific:')
pprint(res_intent_unspecific, width=120, compact=True)

Correct:
{'additional_details': {'actual_user_intent': 'request for current weather in Seattle',
                        'agent_perceived_intent': 'request for current weather in Seattle',
                        'conversation_has_intent': True,
                        'correct_intent_detected': True,
                        'intent_resolved': True},
 'intent_resolution': 5.0,
 'intent_resolution_reason': "The response directly answers the user's query by providing the current weather "
                             "condition and temperature in Seattle, fully addressing the user's request for weather "
                             'information.',
 'intent_resolution_result': 'pass',
 'intent_resolution_threshold': 3}

Incorrect:
{'additional_details': {'actual_user_intent': 'know the weather in Seattle',
                        'agent_perceived_intent': 'provide current weather information for Seattle',
                        'conversation_has_intent': True,
                        'c

### Tool Call Accuracy

Evaluates the agent’s ability to select the appropriate tools, and process correct parameters from previous steps.

In [7]:
tool_definitions = [
    {
        'name': 'fetch_weather',
        'description': 'Fetches weather information for a location.',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {'type': 'string'}
            }
        }
    },
    {
        'name': 'fetch_funfact',
        'description': 'Fetches a fun fact about a location.',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {'type': 'string'}
            }
        }
    }
]


tool_calls_correct = [
    {'type': 'tool_call', 'tool_call_id': 'call_1', 'name': 'fetch_weather', 'arguments': {'location': 'Seattle'}},
]
tool_calls_incorrect = [
    {'type': 'tool_call', 'tool_call_id': 'bad_call', 'name': 'fetch_funfact', 'arguments': {'location': 'Tokyo'}},
]

res_tool_correct = tool_eval(query=user_question, tool_calls=tool_calls_correct, tool_definitions=tool_definitions)
res_tool_incorrect = tool_eval(query=user_question, tool_calls=tool_calls_incorrect, tool_definitions=tool_definitions)


print('Correct:')
pprint(res_tool_correct, width=120, compact=True)
print('\nIncorrect:')
pprint(res_tool_incorrect, width=120, compact=True)

Correct:
{'per_tool_call_details': [],
 'tool_call_accuracy': 'not applicable',
 'tool_call_accuracy_reason': 'Tool call accuracy evaluation is not yet supported for the invoked tools.',
 'tool_call_accuracy_result': 'not applicable',
 'tool_call_accuracy_threshold': 0.8}

Incorrect:
{'per_tool_call_details': [],
 'tool_call_accuracy': 'not applicable',
 'tool_call_accuracy_reason': 'Tool call accuracy evaluation is not yet supported for the invoked tools.',
 'tool_call_accuracy_result': 'not applicable',
 'tool_call_accuracy_threshold': 0.8}


### Task Adherence

 Measures how well the agent’s final response adheres to its assigned tasks, according to its system message and prior steps.


In [8]:
res_task_correct = task_eval(query=user_question, response=agent_response_correct, tool_calls=tool_calls_correct)
res_task_incorrect = task_eval(query=user_question, response=agent_response_incorrect, tool_calls=tool_calls_incorrect)
res_task_unspecific = task_eval(query=user_question_unspecific, response=agent_response_incorrect, tool_calls=tool_calls_incorrect)
print('Correct:')
pprint(res_task_correct, width=120, compact=True)
print('\nIncorrect:')
pprint(res_task_incorrect, width=120, compact=True)
print('\nUnspecific:')
pprint(res_task_unspecific, width=120, compact=True)

Correct:
{'task_adherence': 5.0,
 'task_adherence_reason': "The response is clear, accurate, and directly answers the query about Seattle's weather, "
                          'fulfilling the task completely without any gaps or errors.',
 'task_adherence_result': 'pass',
 'task_adherence_threshold': 3}

Incorrect:
{'task_adherence': 4.0,
 'task_adherence_reason': "The response answers the query about Seattle's weather but includes irrelevant information "
                          'about London, which is unnecessary and detracts from clarity, making it mostly adherent but '
                          'not perfect.',
 'task_adherence_result': 'pass',
 'task_adherence_threshold': 3}

Unspecific:
{'task_adherence': 1.0,
 'task_adherence_reason': 'The response is completely off-topic and does not answer the question about rainfall in '
                          'Spain, thus it is fully inadherent to the task.',
 'task_adherence_result': 'fail',
 'task_adherence_threshold': 3}


## Summary
- **Intent Resolution** confirmed the agent understood the request in both scenarios.
- **Tool Call Accuracy** detected the incorrect tool usage in the flawed scenario.
- **Task Adherence** showed the agent followed instructions even when factual output was wrong.

Use these insights to improve your agent’s tool selection logic and answer verification. For production, combine these metrics with others (e.g., factuality, safety) for a complete quality picture.